Reference: https://github.com/microsoft/dowhy/blob/master/docs/source/example_notebooks/dowhy_estimation_methods.ipynb

## 0. Preparation

In [ ]:
#!pip install dowhy

## 1.0 librariy setting

In [ ]:
import numpy as np
import pandas as pd
import logging

import dowhy
from dowhy import CausalModel
import dowhy.datasets 

## 1.1 Preparation data

ref. https://econml.azurewebsites.net/spec/api.html

In [ ]:
data = dowhy.datasets.linear_dataset(beta=5, ## 因果効果の設定
        num_common_causes=5,    ## 交絡因子(共変量)の数 W0, W1, .., W4
#        num_instruments = 2,    ## 操作変数(原因への変数) Z0, Z1 
        num_treatments=1,       ## 介入変数 v0
        num_samples=10000,      ## Sample numbers
        num_effect_modifiers=3, ## 交互作用の数 X0, X1, ..
        treatment_is_binary=True,
        outcome_is_binary=False,
        stddev_treatment_noise=10)
df = data["df"]
df

## 1.2 Confirm Average Treatment Efffect(ATE)

In [ ]:
data['ate']

## 1.3 Define Causal model

In [ ]:
# With graph
model=CausalModel(
        data = df,
        treatment=data["treatment_name"],     #v0
        outcome=data["outcome_name"],         #y
        graph=data["gml_graph"],
        instruments=data["instrument_names"]
        )

In [ ]:
model.view_model()
#from IPython.display import Image, display
#display(Image(filename="causal_model.png"))

In [ ]:
identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
print(identified_estimand)

## 1.5 Calculate causal effect

## 1.5.1 Causal effect with linear regression

In [ ]:
causal_estimate_reg = model.estimate_effect(identified_estimand,
        method_name="backdoor.linear_regression",
        test_significance=True)
print(causal_estimate_reg)
print("Causal Estimate is " + str(causal_estimate_reg.value))

## 1.5.2 Causal effect with distance matching

In [ ]:
causal_estimate_dmatch = model.estimate_effect(identified_estimand,
                                              method_name="backdoor.distance_matching",
                                              target_units="att",
                                              method_params={'distance_metric':"minkowski", 'p':2})
print(causal_estimate_dmatch)
print("Causal Estimate is " + str(causal_estimate_dmatch.value))

## 1.5.3 Causal effect with propensity score stratification

In [ ]:
causal_estimate_strat = model.estimate_effect(identified_estimand,
                                              method_name="backdoor.propensity_score_stratification",
                                              target_units="att")
print(causal_estimate_strat)
print("Causal Estimate is " + str(causal_estimate_strat.value))

## 1.5.4 Causal effect with propensity matching

In [ ]:
causal_estimate_match = model.estimate_effect(identified_estimand,
                                              method_name="backdoor.propensity_score_matching",
                                              target_units="atc")
print(causal_estimate_match)
print("Causal Estimate is " + str(causal_estimate_match.value))